<h2 style="color:MediumSeaGreen;">Comparing different quantum architectures at the CHSH game</h2>

In [ ]:
import numpy as np
import math

theta_0 = math.pi/8
theta_1 = -math.pi/8

A_0 = np.array([[1, 0], [0, 1]])
A_1 = np.array([[1, 1], [1, -1]]) / math.sqrt(2)

#In numpy, you can take cosine and sine of a number x as "np.cos(x)", "np.sin(x)".
#The syntax for defining your matrix matrices should be, e.g.
#B_0 = np.array([[a11, a12], [a21, a22]])
#where a11, a12, a21, a22 are the entries.
B_0 = np.array([[np.cos(theta_0), np.sin(theta_0)], [-np.sin(theta_0), np.cos(theta_0)]])
B_1 = np.array([[np.cos(theta_1), np.sin(theta_1)], [-np.sin(theta_1), np.cos(theta_1)]])

In [ ]:
from qiskit import QuantumCircuit, transpile
from qiskit_aer import AerSimulator
import random

N = 1000 #number of games played

games_won = 0
sim = AerSimulator()

for i in range(N):              #We will play the game N times
    
    circ = QuantumCircuit(2, 2) #Define a quantum circuit 'circ' on two qubits that creates an EPR pair
    circ.h(0)
    circ.cx(0, 1)    
    
    x = random.randint(0,1)      #sampling a question for Alice
    y = random.randint(0,1)      #sampling a question for Bob
    
    if x == 0:                   
        circ.unitary(A_0, [0], label = 'A_0') #add to the previous circuit the appropriate unitary that Alice applies on question x=0
    else:                        
        circ.unitary(A_1, [0], label = 'A_1') #add to the previous circuit the appropriate unitary that Alice applies on question x=1
        
    if y == 0:       #add Bob's unitary gate B_0, which you have defined in the previous cell (this is already added for you)
        circ.unitary(B_0, [1], label = 'B_0')
    else:            #add Bob's unitary gate B_1, which you have defined in the previous cell (this is already added for you)
        circ.unitary(B_1, [1], label = 'B_1')
    
    
    #add a measurement of both qubits to your circuit
    circ.measure([0, 1], [0, 1])
    
    #compile circuit in terms of IBM's native gates.
    transp_circ = transpile(circ, basis_gates=['rx', 'ry', 'cx'])
    
    #execute ONE SHOT of your circuit transp_circ using IBM's "AerSimulator". 
    job = sim.run(transp_circ, shots=1)

    #get the result of the simulation
    sim_result = job.result()
    
    #get the outcomes
    counts = sim_result.get_counts(transp_circ)
    a = int(list(dict.keys(counts))[0][0])      # Alice's answer
    b = int(list(dict.keys(counts))[0][1])      # Bob's answer

    if (a^b) == (x & y): #check the winning condition, and keep track of the information of whether they won or lost
        games_won += 1
    
#print the ratio of games that were won to the total games played N
print("Ratio of games won to total games played: ", games_won/N)

In [ ]:
#your code here
from qiskit import QuantumCircuit, transpile
from qiskit_aer import AerSimulator

#Define the simulator
sim = AerSimulator()

#Define the number of shots per question
shots = 250

#Initialize the dictionary to store the results
results = {}

#Iterate over all possible questions for Alice and Bob
for x in [0, 1]:
    for y in [0, 1]:
        #initialize the count of games won for this pair of questions
        games_won = 0

        #Play the game 250 times
        for i in range(shots):
            #Define the quantum circuit
            circ = QuantumCircuit(2, 2)
            circ.h(0)
            circ.cx(0, 1)
            if x == 0:
                circ.unitary(A_0, [0], label = 'A_0')
            else:
                circ.unitary(A_1, [0], label = 'A_1')
            if y == 0:
                circ.unitary(B_0, [1], label = 'B_0')
            else:
                circ.unitary(B_1, [1], label = 'B_1')
            circ.measure([0, 1], [0, 1])

            #Compile the circuit
            transp_circ = transpile(circ, basis_gates=['rx', 'ry', 'cx'])
            job = sim.run(transp_circ, shots=1)
            sim_result = job.result()
            counts = sim_result.get_counts(transp_circ)
            a = int(list(dict.keys(counts))[0][0])
            b = int(list(dict.keys(counts))[0][1])

            #Check if the winning condition is satisfied
            if (a^b) == (x & y):
                games_won += 1

        #Store the results in the dictionary
        results[(x, y)] = games_won/shots

#Print the results
for (x, y) in results:
    print("Alice's question: ", x, ", Bob's question: ", y, ", Ratio of games won: ", results[(x, y)])

In [ ]:
from qbraid import get_devices

get_devices()

In [ ]:
from qbraid.providers import QbraidProvider

token = "YOUR_TOKEN" #replace "
provider = QbraidProvider(qiskit_ibm_token=token)
qbraid_id = 'ibm_q_brisbane'
device = provider.get_device(qbraid_id)
job = device.run(circ, shots=250)